In [68]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import statsmodels.formula.api as smf 
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split 
import itertools
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import make_scorer, r2_score
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [69]:
model_gen = '3rd'

In [70]:
def rmse(y_true,y_pred):
    rmse = np.sqrt(mean_squared_error(y_true,y_pred))
    print('rmse',rmse)
    return rmse

# LowPrice Small size models

In [71]:
model_name = 'LowPrice_Smallsize'

In [72]:
df_ml = pd.read_csv('../../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [73]:
df_temp = pd.read_csv('variables_union'+ model_name +'.csv')

In [74]:
target = 'TARGET_Weighted_RetailPrice'

In [75]:
featureimp_variables = list(df_temp['0'].values)

In [76]:
# 6. transform data
data = df_ml.copy()

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + ' + '.join(featureimp_variables) 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                 3372
Model:                                     GLM   Df Residuals:                     3167
Model Family:                            Gamma   Df Model:                          204
Link Function:                             log   Scale:                       0.0014553
Method:                                   IRLS   Log-Likelihood:                -27798.
Date:                         Mon, 10 Jul 2023   Deviance:                       4.5652
Time:                                 02:56:22   Pearson chi2:                     4.61
No. Iterations:                            100   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  8.5462      0.079    108.237      0.000       8.391       8.701
SPEC_AC1socket[T.True]                                     0.0603      0.016      3.883      0.000       0.030       0.091
SPEC_luxurytrim_instrumentpanel_leather                   -0.0127      0.008     -1.674      0.094      -0.027       0.002
SPEC_displays_pressure                                     0.0009      0.003      0.315      0.753      -0.005       0.007
SPEC_operates_above_130_kph_78_mph                        -0.0220      0.005     -4.280      0.000      -0.032      -0.012
SPEC_multi_touch_screen136                                 0.0181      0.005      3.822      0.000       0.009       0.027
SPEC_activates_steering                                    0.0047      0.004      1.072      0.284      -0.004       0.013
SPEC_luxurytrim_handbrakegrip_alloy                        0.0064      0.006      1.155      0.248      -0.004       0.017
SPEC_NHTSA_overall_5                                      -0.0068      0.002     -2.722      0.006      -0.012      -0.002
SPEC_Differential_lock                                     0.0364      0.004      8.808      0.000       0.028       0.045
SPEC_Sunroof                                               0.0902      0.015      6.030      0.000       0.061       0.120
SPEC_pedestrian_collision_mitigation_steering              0.0262      0.009      3.022      0.003       0.009       0.043
SPEC_includes_memory_seat_adjustments                     -0.0251      0.005     -4.585      0.000      -0.036      -0.014
SPEC_headroom_front_mm                                     0.0006   4.84e-05     12.309      0.000       0.001       0.001
SPEC_Charging_connector                                    0.0675      0.011      6.061      0.000       0.046       0.089
SPEC_Remote_services                                      -0.0189      0.010     -1.883      0.060      -0.039       0.001
SPEC_responsivesuspention_electric                         0.0478      0.007      6.375      0.000       0.033       0.062
SPEC_beep_flash                                            0.0134      0.006      2.218      0.027       0.002       0.025
SPEC_FRDRseat_lumbar_memory                                0.0158      0.009      1.781      0.075      -0.002       0.033
SPEC_low_high_gear_ranges                                  0.0420      0.019      2.264      0.024       0.006       0.078
SPEC_BelowseatStorage_passenger                            0.0817      0.008      9.666      0.000       0.065       0.098
SPEC_fold_flat_to_floor         

In [77]:
df_pvalues = pd.DataFrame(result.pvalues,columns=['pvalues'])
significant_values = list(df_pvalues[df_pvalues['pvalues']<0.05].index)

In [78]:
significant_values.remove('Intercept')
try:
    significant_values.remove('SPEC_AC1socket[T.True]')
    significant_values.append('SPEC_AC1socket')
except:
    pass
try:
    significant_values.remove('SPEC_DC1socket[T.True]')
    significant_values.append('SPEC_DC1socket')
except:
    pass

In [79]:
len(significant_values)

134

In [80]:
significant_values_backup = significant_values.copy()

In [81]:
aic_dict = {}
selected_var = []
min_aic = 9999999999999999
for i in range(len(significant_values)):
    updated = 0
    bestaic_var = ''
    for item in significant_values:
        model_combi = selected_var + [item]

        formula = "TARGET_Weighted_RetailPrice ~ 1 +" + '+'.join(model_combi)  
        link = sm.genmod.families.links.log()
        family = sm.families.Gamma(link = link)
        model = smf.glm(formula=formula, data=data, family=family) 
        result = model.fit() 
        
        aic_dict[str('+'.join(model_combi))] = result.aic
        if result.aic < min_aic:
            min_aic = result.aic
            updated += 1
            bestaic_var = item

    if updated > 0:
        selected_var.append(bestaic_var)
        significant_values.remove(bestaic_var)
    else:
        break

    print(i, bestaic_var)

0 MSRP_prevMY_WeightedByVol
1 SPEC_Steering_plastic
2 SPEC_maximum_torque_Nm
3 SPEC_traffic_info
4 SPEC_doormirrorDRside_heater
5 SPEC_published_kerb_weight_kg
6 SPEC_Overtaking_sensor
7 SPEC_Airconditioningtype_manual
8 SPEC_Start_Stop
9 SPEC_Cruise_control
10 SPEC_BelowseatStorage_passenger
11 SPEC_subwoofer
12 SPEC_Alternate_suspension
13 SPEC_ManualTransmission
14 SPEC_Charging_connector
15 SPEC_NHTSA_frontcrash_3
16 SPEC_distance_km1011
17 SPEC_FRPGseat_heated
18 SPEC_headroom_front_mm
19 SPEC_Driver_modes
20 SPEC_LED_highbeam
21 SPEC_entertainmentscreen_biggestsize
22 SPEC_satellite_radio
23 SPEC_Differential_lock
24 SPEC_Steering_alloy
25 SPEC_painted_callipers
26 SPEC_Sunroof
27 SPEC_RR_suspension_multilink
28 SPEC_Capless_fueling_system
29 SPEC_capacity_l289
30 SPEC_luxurytrim_centerconsole_wood
31 SPEC_Front_fog_lights
32 SPEC_responsivesuspention_electric
33 SPEC_IIHS_smalloverlapFRDR_Acceptable
34 SPEC_Secondary_ventilation_controls
35 SPEC_Parking_distance_system
36 SPEC_s

In [82]:
pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').\
    to_csv('AIC_result_' + model_name + '.csv')

In [83]:
best_aic_model = pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').index[0]

In [84]:
X = df_ml.drop(target, axis=1)
y = df_ml[[target]]
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [85]:
# 6. transform data
data = pd.concat([train_X[best_aic_model.split('+')], train_y], axis=1)

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + best_aic_model 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                 2360
Model:                                     GLM   Df Residuals:                     2234
Model Family:                            Gamma   Df Model:                          125
Link Function:                             log   Scale:                       0.0014968
Method:                                   IRLS   Log-Likelihood:                -19497.
Date:                         Mon, 10 Jul 2023   Deviance:                       3.3062
Time:                                 03:15:52   Pearson chi2:                     3.34
No. Iterations:                             10   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         8.5940      0.061    139.907      0.000       8.474       8.714
SPEC_AC1socket[T.True]                            0.0568      0.018      3.245      0.001       0.022       0.091
MSRP_prevMY_WeightedByVol                      1.478e-05   6.23e-07     23.740      0.000    1.36e-05     1.6e-05
SPEC_Steering_plastic                            -0.0275      0.003     -8.310      0.000      -0.034      -0.021
SPEC_maximum_torque_Nm                            0.0005    3.2e-05     16.212      0.000       0.000       0.001
SPEC_traffic_info                                 0.0256      0.005      4.995      0.000       0.016       0.036
SPEC_doormirrorDRside_heater                      0.0076      0.003      2.768      0.006       0.002       0.013
SPEC_published_kerb_weight_kg                     0.0003   1.63e-05     20.419      0.000       0.000       0.000
SPEC_Overtaking_sensor                            0.0229      0.003      8.238      0.000       0.017       0.028
SPEC_Airconditioningtype_manual                  -0.0426      0.003    -13.270      0.000      -0.049      -0.036
SPEC_Start_Stop                                   0.0200      0.003      6.024      0.000       0.013       0.026
SPEC_Cruise_control                               0.0406      0.005      7.764      0.000       0.030       0.051
SPEC_BelowseatStorage_passenger                   0.0871      0.009      9.736      0.000       0.070       0.105
SPEC_subwoofer                                    0.0357      0.003     10.550      0.000       0.029       0.042
SPEC_Alternate_suspension                         0.0264      0.004      7.294      0.000       0.019       0.033
SPEC_ManualTransmission                          -0.0458      0.003    -15.775      0.000      -0.051      -0.040
SPEC_Charging_connector                           0.0715      0.013      5.630      0.000       0.047       0.096
SPEC_NHTSA_frontcrash_3                          -0.0625      0.007     -9.282      0.000      -0.076      -0.049
SPEC_distance_km1011                          -3.318e-07   5.22e-08     -6.354      0.000   -4.34e-07   -2.29e-07
SPEC_FRPGseat_heated                              0.0285      0.003      9.995      0.000       0.023       0.034
SPEC_headroom_front_mm                            0.0006   4.96e-05     11.359      0.000       0.000       0.001
SPEC_Driver_modes                                 0.0258      0.007      3.627      0.000       0.012       0.040
SPEC_LED_highbeam                                 0.0187      0.003      6.338      0.000       0.013       0.024
SPEC_entertainmentsc

In [86]:
pred_y = result.predict(test_X)

In [88]:
rmse(pred_y,test_y['TARGET_Weighted_RetailPrice'].values)

rmse 1117.9495624291649


1117.9495624291649

In [89]:
r2_score(pred_y,test_y['TARGET_Weighted_RetailPrice'].values)

0.9590406971814744

# MidPrice_Midsize models

In [90]:
model_name = 'MidPrice_Midsize'

In [91]:
df_ml = pd.read_csv('../../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [92]:
target = 'TARGET_Weighted_RetailPrice'

In [93]:
featureimp_variables = list(df_ml.columns)
featureimp_variables.remove(target)

In [94]:
# 6. transform data
data = df_ml.copy()

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + ' + '.join(featureimp_variables) 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                 4514
Model:                                     GLM   Df Residuals:                     4286
Model Family:                            Gamma   Df Model:                          227
Link Function:                             log   Scale:                       0.0019518
Method:                                   IRLS   Log-Likelihood:                -40306.
Date:                         Mon, 10 Jul 2023   Deviance:                       8.3222
Time:                                 08:40:16   Pearson chi2:                     8.37
No. Iterations:                             31   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                 10.7241      0.142     75.724      0.000      10.447      11.002
SPEC_cab_type_none                                        -0.0190      0.012     -1.526      0.127      -0.043       0.005
SPEC_Steering_alloy                                        0.0092      0.003      3.456      0.001       0.004       0.014
SPEC_responsivesuspention_mechanical                       0.0276      0.007      3.989      0.000       0.014       0.041
SPEC_driver_monitor                                        0.0069      0.003      2.276      0.023       0.001       0.013
SPEC_doormirrorPGside_electricadjust                       0.0181      0.028      0.645      0.519      -0.037       0.073
SPEC_Rear_side_windows                                     0.0007      0.008      0.080      0.936      -0.016       0.017
SPEC_multi_touch_screen136                                -0.0028      0.004     -0.754      0.451      -0.010       0.005
SPEC_Front_fog_lights                                     -0.0006      0.003     -0.206      0.837      -0.006       0.005
SPEC_Panoramic_roof                                       -0.0021      0.005     -0.437      0.662      -0.011       0.007
SPEC_USB_connection_front                                  0.0375      0.006      5.966      0.000       0.025       0.050
SPEC_Remote_engine_starter                                -0.0122      0.003     -3.983      0.000      -0.018      -0.006
SPEC_LED_breaklight                                        0.0187      0.005      3.761      0.000       0.009       0.029
SPEC_touch_controls                                        0.0514      0.010      5.218      0.000       0.032       0.071
SPEC_on_side_of_body                                       0.0121      0.003      3.853      0.000       0.006       0.018
SPEC_HeadUpDisplay                                         0.0042      0.004      1.046      0.296      -0.004       0.012
SPEC_includes_augmented_reality                           -0.0060      0.035     -0.171      0.864      -0.075       0.063
SPEC_RRseatbelt_heightadjust                               0.0228      0.007      3.142      0.002       0.009       0.037
SPEC_4WD_fulltime                                          0.0282      0.002     14.068      0.000       0.024       0.032
SPEC_screen_size_in                                        0.0033      0.000     11.840      0.000       0.003       0.004
MSRP_prevMY_WeightedByVol                               7.353e-06   2.55e-07     28.780      0.000    6.85e-06    7.85e-06
SPEC_Mudflaps_FR                

In [95]:
df_pvalues = pd.DataFrame(result.pvalues,columns=['pvalues'])
significant_values = list(df_pvalues[df_pvalues['pvalues']<0.05].index)

In [96]:
significant_values.remove('Intercept')
try:
    significant_values.remove('SPEC_AC1socket[T.True]')
    significant_values.append('SPEC_AC1socket')
except:
    pass
try:
    significant_values.remove('SPEC_DC1socket[T.True]')
    significant_values.append('SPEC_DC1socket')
except:
    pass

In [97]:
len(significant_values)

142

In [98]:
significant_values_backup = significant_values.copy()

In [99]:
aic_dict = {}
selected_var = []
min_aic = 9999999999999999
for i in range(len(significant_values)):
    updated = 0
    bestaic_var = ''
    for item in significant_values:
        model_combi = selected_var + [item]

        formula = "TARGET_Weighted_RetailPrice ~ 1 +" + '+'.join(model_combi)  
        link = sm.genmod.families.links.log()
        family = sm.families.Gamma(link = link)
        model = smf.glm(formula=formula, data=data, family=family) 
        result = model.fit() 
        
        aic_dict[str('+'.join(model_combi))] = result.aic
        if result.aic < min_aic:
            min_aic = result.aic
            updated += 1
            bestaic_var = item

    if updated > 0:
        selected_var.append(bestaic_var)
        significant_values.remove(bestaic_var)
    else:
        break

    print(i, bestaic_var)

0 MSRP_prevMY_WeightedByVol
1 SPEC_maximum_torque_Nm
2 SPEC_Steering_plastic
3 SPEC_FRDRseat_ventilated
4 SPEC_doormirror_autodimming
5 SPEC_tire_diameter_inch
6 SPEC_low_high_gear_ranges
7 SPEC_4WD_fulltime
8 SPEC_wheelrim_width_inch
9 SPEC_Ceramic_brakes
10 SPEC_Warranty_battery
11 SPEC_Electrically_heated_windshield
12 SPEC_FRDRseat_lumbar_electric
13 SPEC_doormirrorPGside_tiltforreverse
14 SPEC_screen_size_in
15 SPEC_Secondary_ventilation_controls
16 SPEC_FRPGseat_lumbar_electric
17 SPEC_cc
18 SPEC_shoulder_room_front_mm
19 SPEC_Electric_trunk_hatch_pull_down
20 SPEC_instantaneous_fuel_consumption
21 SPEC_published_kerb_weight_kg
22 SPEC_parkingdistsys_FRRRSide
23 SPEC_luxurytrim_centerconsole_suede
24 SPEC_Mudflaps_RR
25 SPEC_Overtaking_sensor
26 NCBS_OVERALL_Brand_Image
27 SPEC_Charges
28 SPEC_LED_breaklight
29 SPEC_Differential_lock
30 SPEC_NHTSA_frontcrash_4
31 SPEC_Spare_wheel
32 SPEC_in_dash_DVD
33 SPEC_Power_consumption
34 SPEC_USB_connection_front
35 NCBS_OVERALL_Power_And_

In [100]:
pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').\
    to_csv('AIC_result_' + model_name + '.csv')

In [101]:
best_aic_model = pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').index[0]

In [102]:
X = df_ml.drop(target, axis=1)
y = df_ml[[target]]
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [103]:
# 6. transform data
data = pd.concat([train_X[best_aic_model.split('+')], train_y], axis=1)

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + best_aic_model 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                 3159
Model:                                     GLM   Df Residuals:                     3020
Model Family:                            Gamma   Df Model:                          138
Link Function:                             log   Scale:                       0.0019940
Method:                                   IRLS   Log-Likelihood:                -28257.
Date:                         Mon, 10 Jul 2023   Deviance:                       6.0046
Time:                                 09:48:52   Pearson chi2:                     6.02
No. Iterations:                            100   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
====================================================================================================================
                                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           10.7229      0.127     84.180      0.000      10.473      10.973
MSRP_prevMY_WeightedByVol                          7.76e-06   2.57e-07     30.140      0.000    7.26e-06    8.26e-06
SPEC_maximum_torque_Nm                               0.0006    2.5e-05     25.292      0.000       0.001       0.001
SPEC_Steering_plastic                               -0.0299      0.005     -6.580      0.000      -0.039      -0.021
SPEC_FRDRseat_ventilated                             0.0291      0.009      3.156      0.002       0.011       0.047
SPEC_doormirror_autodimming                          0.0190      0.003      6.006      0.000       0.013       0.025
SPEC_tire_diameter_inch                              0.0072      0.001      8.593      0.000       0.006       0.009
SPEC_low_high_gear_ranges                            0.0619      0.005     12.375      0.000       0.052       0.072
SPEC_4WD_fulltime                                    0.0289      0.002     12.666      0.000       0.024       0.033
SPEC_wheelrim_width_inch                             0.0330      0.003     11.795      0.000       0.027       0.038
SPEC_Ceramic_brakes                                  0.3639      0.027     13.267      0.000       0.310       0.418
SPEC_Warranty_battery                                0.0356      0.005      7.395      0.000       0.026       0.045
SPEC_Electrically_heated_windshield                  0.0936      0.014      6.734      0.000       0.066       0.121
SPEC_FRDRseat_lumbar_electric                        0.0208      0.003      6.201      0.000       0.014       0.027
SPEC_doormirrorPGside_tiltforreverse                 0.0368      0.004      8.565      0.000       0.028       0.045
SPEC_screen_size_in                                  0.0035      0.000     13.316      0.000       0.003       0.004
SPEC_Secondary_ventilation_controls                  0.0184      0.005      3.908      0.000       0.009       0.028
SPEC_FRPGseat_lumbar_electric                        0.0228      0.003      8.258      0.000       0.017       0.028
SPEC_cc                                           2.978e-05   3.41e-06      8.732      0.000    2.31e-05    3.65e-05
SPEC_shoulder_room_front_mm                         -0.0008   5.03e-05    -15.554      0.000      -0.001      -0.001
SPEC_Electric_trunk_hatch_pull_down                  0.0262      0.003      8.876      0.000       0.020       0.032
SPEC_instantaneous_fuel_consumption                  0.0231      0.004      6.166      0.000       0.016       0.030
SPEC_published_kerb_weight_kg                        0.0002

In [104]:
pred_y = result.predict(test_X)

In [105]:
rmse(pred_y,test_y['TARGET_Weighted_RetailPrice'].values)

rmse 2479.7554963899643


2479.7554963899643

In [106]:
r2_score(pred_y,test_y['TARGET_Weighted_RetailPrice'].values)

0.9604190509683439

# MidPrice_Largesize models

In [107]:
model_name = 'MidPrice_Largesize'

In [108]:
df_ml = pd.read_csv('../../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [109]:
target = 'TARGET_Weighted_RetailPrice'

In [110]:
featureimp_variables = list(df_ml.columns)
featureimp_variables.remove(target)

In [111]:
# 6. transform data
data = df_ml.copy()

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + ' + '.join(featureimp_variables) 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                 3822
Model:                                     GLM   Df Residuals:                     3607
Model Family:                            Gamma   Df Model:                          214
Link Function:                             log   Scale:                      0.00086211
Method:                                   IRLS   Log-Likelihood:                -32363.
Date:                         Mon, 10 Jul 2023   Deviance:                       3.1128
Time:                                 09:49:09   Pearson chi2:                     3.11
No. Iterations:                            100   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  0.2019      0.003     65.837      0.000       0.196       0.208
SPEC_cab_type_none                                        -0.0263      0.012     -2.247      0.025      -0.049      -0.003
SPEC_Steering_alloy                                       -0.0459      0.007     -6.991      0.000      -0.059      -0.033
SPEC_stroke                                                0.0026      0.000      9.135      0.000       0.002       0.003
SPEC_digital_media_card                                   -0.0083      0.003     -2.534      0.011      -0.015      -0.002
SPEC_Bumper_FRRR                                          -0.0039      0.004     -0.897      0.370      -0.012       0.005
SPEC_Rear_side_windows                                     0.0214      0.005      3.973      0.000       0.011       0.032
SPEC_Compressor_turbo                                      0.0728      0.005     13.622      0.000       0.062       0.083
SPEC_LED_lights                                            0.0057      0.003      1.699      0.089      -0.001       0.012
SPEC_Service_interval_indicator                           -0.0235      0.006     -3.643      0.000      -0.036      -0.011
SPEC_Front_fog_lights                                      0.0120      0.003      4.183      0.000       0.006       0.018
SPEC_Trailer_towing_mirrors                               -0.0254      0.006     -4.112      0.000      -0.038      -0.013
SPEC_Cargo_area_light119                                   0.0417      0.005      8.687      0.000       0.032       0.051
PROFILE_Make_FORD                                          0.0025      0.008      0.312      0.755      -0.013       0.018
SPEC_includes_walk_away_locking                            0.0085      0.009      0.893      0.372      -0.010       0.027
SPEC_Rear_cross_traffic_alert                             -0.0749      0.016     -4.824      0.000      -0.105      -0.044
SPEC_auto_dimming                                         -0.0128      0.004     -2.924      0.003      -0.021      -0.004
SPEC_Emergency_call                                       -0.0768      0.009     -8.945      0.000      -0.094      -0.060
SPEC_LED_breaklight                                       -0.0412      0.007     -6.326      0.000      -0.054      -0.028
SPEC_motortorqueNm_max                                     0.0116      0.062      0.185      0.853      -0.111       0.134
SPEC_HeadUpDisplay                                         0.0873      0.019      4.484      0.000       0.049       0.125
SPEC_RRseatbelt_heightadjust    

In [112]:
df_pvalues = pd.DataFrame(result.pvalues,columns=['pvalues'])
significant_values = list(df_pvalues[df_pvalues['pvalues']<0.05].index)

In [113]:
significant_values.remove('Intercept')
try:
    significant_values.remove('SPEC_AC1socket[T.True]')
    significant_values.append('SPEC_AC1socket')
except:
    pass
try:
    significant_values.remove('SPEC_DC1socket[T.True]')
    significant_values.append('SPEC_DC1socket')
except:
    pass

In [114]:
len(significant_values)

167

In [115]:
significant_values_backup = significant_values.copy()

In [116]:
aic_dict = {}
selected_var = []
min_aic = 9999999999999999
for i in range(len(significant_values)):
    updated = 0
    bestaic_var = ''
    for item in significant_values:
        model_combi = selected_var + [item]

        formula = "TARGET_Weighted_RetailPrice ~ 1 +" + '+'.join(model_combi)  
        link = sm.genmod.families.links.log()
        family = sm.families.Gamma(link = link)
        model = smf.glm(formula=formula, data=data, family=family) 
        result = model.fit() 
        
        aic_dict[str('+'.join(model_combi))] = result.aic
        if result.aic < min_aic:
            min_aic = result.aic
            updated += 1
            bestaic_var = item

    if updated > 0:
        selected_var.append(bestaic_var)
        significant_values.remove(bestaic_var)
    else:
        break

    print(i, bestaic_var)

0 MSRP_prevMY_WeightedByVol
1 SPEC_FRseatbelt_Center
2 SPEC_Number_of_doors32
3 SPEC_FRDRseat_ventilated
4 SPEC_Drivenwheels_rear
5 SPEC_length_x_width
6 SPEC_wheelrim_alloy
7 SPEC_compression_ratio
8 MACRO_Inflation
9 SPEC_Anti_collision_system
10 SPEC_injection_gasoline_direct_injection
11 SPEC_cab_crew
12 SPEC_height_mm
13 SPEC_Rear_side_windows
14 SPEC_step_retractable
15 SPEC_dutyrating_one_ton
16 SPEC_dutyrating_class_4
17 SPEC_Steering_plastic
18 SPEC_dusk_sensor
19 SPEC_Glass_roof
20 SPEC_Trailer_assist
21 SPEC_subwoofer
22 SPEC_4X4_specifics
23 SPEC_Front_fog_lights
24 SPEC_Compressor_turbo
25 SPEC_bore
26 SPEC_Power_windows
27 SPEC_CountryOfAssembly_USA
28 SPEC_FRPGseat_tilt_electric
29 SPEC_FRDRseat_reclining_electric
30 SPEC_colour_screen
31 SPEC_low_rolling_resistance
32 SPEC_amount
33 SPEC_headlight_Xenon
34 SPEC_Steering_alloy
35 SPEC_Charges
36 SPEC_FRDRseat_massage
37 SPEC_luxurytrim_instrumentpanel_titanium
38 SPEC_Wheel_opening_moldings
39 SPEC_Steering_wood
40 SPEC_

In [117]:
pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').\
    to_csv('AIC_result_' + model_name + '.csv')

In [118]:
best_aic_model = pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').index[0]

In [119]:
X = df_ml.drop(target, axis=1)
y = df_ml[[target]]
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [120]:
# 6. transform data
data = pd.concat([train_X[best_aic_model.split('+')], train_y], axis=1)

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + best_aic_model 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                 2675
Model:                                     GLM   Df Residuals:                     2534
Model Family:                            Gamma   Df Model:                          140
Link Function:                             log   Scale:                      0.00090063
Method:                                   IRLS   Log-Likelihood:                -22707.
Date:                         Mon, 10 Jul 2023   Deviance:                       2.2813
Time:                                 14:37:10   Pearson chi2:                     2.28
No. Iterations:                            100   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  8.7433      0.058    150.635      0.000       8.630       8.857
MSRP_prevMY_WeightedByVol                               5.903e-06   4.45e-07     13.265      0.000    5.03e-06    6.78e-06
SPEC_FRseatbelt_Center                                    -0.0436      0.004    -11.506      0.000      -0.051      -0.036
SPEC_Number_of_doors32                                    -0.0222      0.006     -3.978      0.000      -0.033      -0.011
SPEC_FRDRseat_ventilated                                  -0.0237      0.008     -2.901      0.004      -0.040      -0.008
SPEC_Drivenwheels_rear                                    -0.1048      0.002    -56.191      0.000      -0.108      -0.101
SPEC_length_x_width                                     2.303e-08   1.22e-09     18.815      0.000    2.06e-08    2.54e-08
SPEC_wheelrim_alloy                                        0.0602      0.003     18.060      0.000       0.054       0.067
SPEC_compression_ratio                                     0.0066      0.002      3.240      0.001       0.003       0.011
MACRO_Inflation                                            0.0037      0.001      6.335      0.000       0.003       0.005
SPEC_Anti_collision_system                                 0.0095      0.008      1.151      0.250      -0.007       0.026
SPEC_injection_gasoline_direct_injection                  -0.0715      0.016     -4.559      0.000      -0.102      -0.041
SPEC_cab_crew                                              0.0704      0.002     30.165      0.000       0.066       0.075
SPEC_height_mm                                          1.338e-05   9.26e-06      1.446      0.148   -4.76e-06    3.15e-05
SPEC_Rear_side_windows                                     0.0186      0.005      3.401      0.001       0.008       0.029
SPEC_step_retractable                                      0.0453      0.007      6.399      0.000       0.031       0.059
SPEC_dutyrating_one_ton                                    0.0447      0.002     18.567      0.000       0.040       0.049
SPEC_dutyrating_class_4                                    0.1147      0.007     16.424      0.000       0.101       0.128
SPEC_Steering_plastic                                     -0.0272      0.004     -6.300      0.000      -0.036      -0.019
SPEC_dusk_sensor                                           0.0115      0.003      3.437      0.001       0.005       0.018
SPEC_Glass_roof                                            0.0887      0.011      8.139      0.000       0.067       0.110
SPEC_Trailer_assist             

In [121]:
pred_y = result.predict(test_X)

In [122]:
rmse(pred_y,test_y['TARGET_Weighted_RetailPrice'].values)

rmse 1401.2142764336447


1401.2142764336447

In [123]:
r2_score(pred_y,test_y['TARGET_Weighted_RetailPrice'].values)

0.9745669556068683